In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import sys
from PIL import Image
import cv2
from datetime import datetime
import argparse

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Dropout, Input, Flatten, Dense, Lambda, Add, Multiply
from keras import backend as K
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam, SGD


In [ ]:
def showImage(image, isGray=False):
    fig = plt.figure(figsize=(10, 10))
    plt.imshow(image, cmap = 'gray')
    plt.show()


In [3]:
'''Orginalne obrazki'''
ROOT_DIR = os.path.dirname(os.path.abspath("top_level_file.txt"))

POINTS_DIR = ROOT_DIR + "/Images/Points/points/*.dat"
IMAGE_DIR = ROOT_DIR + "/Images/orgs/*.bmp"
points = glob.glob(POINTS_DIR)
orgs = glob.glob(IMAGE_DIR)
points.sort()
orgs.sort()

print(len(points))
print(len(orgs))

size = (128,128)

imgs = []
for image in orgs:
    im = cv2.imread(image)
    im = im[:,:,0:1]/255
    imgs.append(im)

imgs = np.asarray(imgs)
#imgs.reshape(128,128,1)

X = []
Y = []
for f in points:
    file = open(f,'r')
    XY_ =np.array([list(map(float,l.split(' ')[1:])) for l in file.read().splitlines()])
    file.close()
    X_ = XY_[:,0].reshape(-1,50)
    Y_ = XY_[:,1].reshape(-1,50)
    X.append(X_)
    Y.append(Y_)
#    print(X_.shape)
#    print(Y_.shape)

X = np.asarray(X,dtype=np.float64)[:,0,:]
Y = np.asarray(Y,dtype=np.float64)[:,0,:]

print(imgs.shape)
print(X.shape)
print(Y.shape)


140
140
(140, 128, 128, 1)
(140, 50)
(140, 50)


In [4]:
N_COMP = 37 #l. punktów po generowaniu PCA
N_FEATURES = 100 #l. wszystkich współrzędnych (2*50)
comp = []
f = open('pca.dat','r')
for row in range(N_COMP):
    for col in range(N_FEATURES):
        comp.append(float(f.readline()))
f.close()
comp = np.asarray(comp,dtype=np.float32).reshape(N_COMP,N_FEATURES)
print(comp.shape)
compX = comp[:,0::2]
compY = comp[:,1::2]
#print(comp)
#print(compX)
#print(compY)

av = []
f = open('mean.dat','r')# średni kształt wyliczony w contours
for row in range(N_FEATURES):
    av.append(float(f.readline()))
f.close()
av = np.asarray(av,dtype=np.float32).reshape(-1,N_FEATURES)
print(av.shape)
avX = av[:,0::2]
avY = av[:,1::2]
#print(av)
#print(avX)
#print(avY)


(37, 100)
(1, 100)


In [5]:
N_COMP = 37
N_FEATURES = 100

def conv2d_block(
    inputs, 
    filters=16, 
    kernel_size=(3,3), 
    activation='relu', 
    kernel_initializer= 'glorot_uniform',
    padding='same'):
    
    c = Conv2D(filters, kernel_size, activation=activation, kernel_initializer=kernel_initializer, padding=padding) (inputs)
    c = Conv2D(filters, kernel_size, activation=activation, kernel_initializer=kernel_initializer, padding=padding) (c)
    return c

def linear(w,comp,av):
    
    b = tf.constant(comp, shape=(N_COMP, N_FEATURES//2))
    c = tf.constant(av, shape=(1, N_FEATURES//2))
    d = tf.matmul(w, b) + c
    return d


NUM_LAYERS = 4
FILTERS = 16
NUM_ELEMENTS = 1000

inputs = Input(size+(1,))

x = inputs   

for l in range(NUM_LAYERS):
    x = conv2d_block(inputs=x, filters=FILTERS)
    x = MaxPooling2D((2, 2)) (x)
    FILTERS = FILTERS*2 

x = Flatten()(x)

x = Dense(NUM_ELEMENTS,activation='relu')(x)
x = Dense(NUM_ELEMENTS,activation='relu')(x)

w = Dense(N_COMP)(x)
A = Dense(1)(x) # współczynniki pierwszego wiersza macierzy homografii (wsp x)
B = Dense(1)(x)
C = Dense(1)(x)
D = Dense(1)(x) #współczynniki dla drugiego wiersza macierzy homografii (wsp y)
E = Dense(1)(x)
F = Dense(1)(x)

'''ta sekcja zostaje zakomentowana'''
# X_ = Lambda(linear,arguments={'comp':compx,'av':avX})(w)#pca.inverse transform+mean
# Y_ = Lambda(linear,arguments={'comp':compY,'av':avY})(w)#pca.inverse transform+mean
#
# outX = Add()([Multiply()([A,X_]),Multiply()([B,Y_]),C])#wynik obliczeń razy macierz homografii
# outY = Add()([Multiply()([D,X_]),Multiply()([E,Y_]),F])#wynik obliczeń razy macierz homografii
''''''
# w-pca.components
# A-
model = Model(inputs=[inputs],outputs=[w,A,B,C,D,E,F])

model.compile(optimizer=Adam(lr=0.01),
                loss='mse',
                metrics=[tf.keras.metrics.MeanSquaredError()]
                 )
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 16) 160         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 128, 128, 16) 2320        conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 64, 64, 16)   0           conv2d_2[0][0]                   
____________________________________________________________________________________________

In [6]:
'''PCA'''
#PCA inverse transform

# TESTED = 163
# r = np.zeros((50,1),dtype=np.float32)
# print(compX.shape)
# for col in range(compX):
#     r[col] = np.sum(compX[:,col]*X_projected[TESTED])
# # print(r-X_recon[TESTED])

'PCA'

In [9]:
X_ = linear(w,compX,avX)
Y_ = linear(w,compY,avY)
outX = tf.math.multiply(A,X_)+tf.math.multiply(B,Y_)+C
outY = tf.math.multiply(D,X_)+tf.math.multiply(E,Y_)+F
print(outY.shape)

(None, 50)


In [10]:
'''procrustes'''

'procrustes'

In [11]:
print(imgs.shape)
history = model.fit(imgs,[w,A,B,C,D,E,F],epochs=100,batch_size=10,validation_split = 0.2)

(140, 128, 128, 1)


ValueError: When feeding symbolic tensors to a model, we expect the tensors to have a static batch size. Got tensor with shape: (None, 37)

In [ ]:
TESTED = 1
yyy = model.predict(imgs[TESTED:TESTED+1])

In [ ]:
print(yyy)

In [ ]:
im = imgs[TESTED,:,:,0].copy()
print(yyy[0].shape)

for p in range(50):
    if int(yyy[1][0,p]) >=128:
        continue
    if int(yyy[0][0,p]) >=128:
        continue
    im[int(yyy[0][0,p]),int(yyy[1][0,p])] = 0


showImage(im)